In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from mymodule import PipeLine


# 使用するデータセット
df = pd.read_csv('./data/train.csv')
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,56,1,ASY,155,342,1,Normal,150,1,3.0,Flat,1
1,55,0,ATA,130,394,0,LVH,150,0,0.0,Up,0
2,47,1,NAP,110,0,1,Normal,120,1,0.0,Flat,1


In [2]:
# 結果を出力する関数
def out_put(model, x_train, x_test, y_train, y_test):
    print('-'*20, '正解率（訓練データ）', '-'*20)
    train_pred = model.predict(x_train)
    print(accuracy_score(y_train, train_pred))
    print('-'*20, '正解率（検証データ）', '-'*20)
    test_pred = model.predict(x_test)
    print(accuracy_score(y_test, test_pred))

>
# ベースラインの作成
ベースラインを作成して今後の特徴作成やモデル選択の比較対象とする<br>
カテゴリ変数と欠損の多いコレステロール値のカラムを除いたものとする
モデルは線形分離モデルのSVCを使用

In [3]:
df = pd.read_csv('./data/train.csv')

# 前処理
pipe = PipeLine()
pipe(df)
pipe.df_num = pipe.df_num.drop(['Cholesterol'], axis=1)
print('-'*20, ' ベースラインで使用する変数', '-'*20)
display(pipe.df_num.head(3))
# 7:3でデータを分割(seed値はpipe.random_seed=scala: intで設定)
pack = pipe.fold_out_split(test_size=0.3)  # x_train, x_test, y_train, y_test = pack

# サポートベクターマシンでモデルを訓練
model = SVC()
model.fit(pack[0], pack[2])

# 予測値を出力
out_put(model, *pack)

--------------------  ベースラインで使用する変数 --------------------


,Age,Sex,RestingBP,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,1,150,1,3.0
1,55,0,130,0,150,0,0.0
2,47,1,110,1,120,1,0.0


-------------------- 分割されたデータShape --------------------
x_train: (449, 7) x_test: (193, 7)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.6926503340757239
-------------------- 正解率（検証データ） --------------------
0.694300518134715


### コレステロール値を追加して比較する
今回は欠損値の補完は考えない<br>
欠損値補完は考察のもと別途考える必要がある

In [4]:
df = pd.read_csv('./data/train.csv')

# 前処理
pipe = PipeLine()
pipe(df)
print('-'*20, ' 変数にコレステロール値を追加', '-'*20)
display(pipe.df_num.head(3))
pack = pipe.fold_out_split(test_size=0.3)  # x_train, x_test, y_train, y_test = pack

# サポートベクターマシンでモデルを訓練
model = SVC()
model.fit(pack[0], pack[2])

# 予測値を出力
out_put(model, *pack)

--------------------  変数にコレステロール値を追加 --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0


-------------------- 分割されたデータShape --------------------
x_train: (449, 8) x_test: (193, 8)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.7505567928730512
-------------------- 正解率（検証データ） --------------------
0.7046632124352331


訓練誤差は5%ほど向上したが検証誤差は1%ほどしか向上してないが改善がみられた

>
### ベースラインの標準化
コレステロールを含めたベースラインの説明変数に標準化を取り入れる

In [5]:
df = pd.read_csv('./data/train.csv')

# 前処理
pipe = PipeLine()
pipe(df)
pipe.standard_scaler()  # 標準化の処理を追加
pack = pipe.fold_out_split(test_size=0.3)

# サポートベクターマシンでモデルを訓練
model = SVC()
model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

# 予測値を出力
out_put(model, *pack)

-------------------- 標準化されたdf_num --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,0.256306,0.520852,1.272672,1.323176,1.742910,0.512265,1.252198,1.959903
1,0.151431,-1.919930,-0.116784,1.796972,-0.573753,0.512265,-0.798596,-0.809353
2,-0.687567,0.520852,-1.228348,-1.792941,1.742910,-0.706992,1.252198,-0.809353


-------------------- 分割されたデータShape --------------------
x_train: (449, 8) x_test: (193, 8)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.8641425389755011
-------------------- 正解率（検証データ） --------------------
0.8238341968911918


標準化を施すことで<span style="color: orange;">ベースラインより精度が10%以上向上</span>していることがわかる
>

# One Hot Encodeを追加（SVCモデル）
ベースラインで取り除いたカテゴリ変数をOne_Hot_Encodingで説明変数に取り入れて比較する<br>
(比較を行うために標準化は行わない)

In [6]:
df = pd.read_csv('./data/train.csv')
# 前処理
pipe = PipeLine()
pipe(df)
# pipeのカテゴリデータを保持したアトリビュートからカラム名を取得
one_hot_columns = pipe.df_cat.columns
pipe.one_hot(one_hot_columns)  # pipeのクラスメソッドでワンホット化
pack = pipe.fold_out_split(test_size=0.3)

# サポートベクターマシンでモデルを訓練
model = SVC()
model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

# 予測値を出力
out_put(model, *pack)

-------------------- ワンホットされたカラムIndex(['ChestPainType', 'RestingECG', 'ST_Slope'], dtype='object') --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0


-------------------- 分割されたデータShape --------------------
x_train: (449, 18) x_test: (193, 18)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.7527839643652561
-------------------- 正解率（検証データ） --------------------
0.6994818652849741


訓練誤差は向上しているが汎化誤差はベースラインとほぼ一緒<br>むしろ悪い
>
以下は各カテゴリを別々で追加して精度を確認

In [7]:
df = pd.read_csv('./data/train.csv')
pipe = PipeLine()
pipe(df)
one_hot_columns = pipe.df_cat.columns  # カテゴリデータのカラムを取得

# それぞれのカラムがどのように影響するか調べていく
for cat in one_hot_columns:
    print('#'*150)
    pipe = PipeLine()  # インスタンスの初期化
    pipe(df) 
    pipe.one_hot(cat)
    pack = pipe.fold_out_split(test_size=0.3)

    # サポートベクターマシンでモデルを訓練
    model = SVC()
    model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

    # 予測値を出力
    out_put(model, *pack)
    print()

######################################################################################################################################################
-------------------- ワンホットされたカラムChestPainType --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ASY,ATA,NAP,TA
0,56,1,155,342,1,150,1,3.0,1,0,0,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0
2,47,1,110,0,1,120,1,0.0,0,0,1,0


-------------------- 分割されたデータShape --------------------
x_train: (449, 12) x_test: (193, 12)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.755011135857461
-------------------- 正解率（検証データ） --------------------
0.6994818652849741

######################################################################################################################################################
-------------------- ワンホットされたカラムRestingECG --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,LVH,Normal,ST
0,56,1,155,342,1,150,1,3.0,0,1,0
1,55,0,130,394,0,150,0,0.0,1,0,0
2,47,1,110,0,1,120,1,0.0,0,1,0


-------------------- 分割されたデータShape --------------------
x_train: (449, 11) x_test: (193, 11)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.7527839643652561
-------------------- 正解率（検証データ） --------------------
0.6994818652849741

######################################################################################################################################################
-------------------- ワンホットされたカラムST_Slope --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,Down,Flat,Up
0,56,1,155,342,1,150,1,3.0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,1,0


-------------------- 分割されたデータShape --------------------
x_train: (449, 11) x_test: (193, 11)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.7527839643652561
-------------------- 正解率（検証データ） --------------------
0.6994818652849741



標準化していないためかカテゴリ変数の効果は薄いよう<br>
モデルの表現力不足のせいかRestingECGとST_Slopeの結果に変化がない

### 標準化とワンホットエンコーディング（SVC)
標準化をしたものにワンホットエンコードがどのように寄与するか確認する

In [8]:
df = pd.read_csv('./data/train.csv')
# 前処理
pipe = PipeLine()
pipe(df)
one_hot_columns = pipe.df_cat.columns
pipe.one_hot(one_hot_columns)
pipe.standard_scaler()  # 標準化の処理を追加
pack = pipe.fold_out_split(test_size=0.3)

# サポートベクターマシンでモデルを訓練
model = SVC()
model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

# 予測値を出力
out_put(model, *pack)

-------------------- ワンホットされたカラムIndex(['ChestPainType', 'RestingECG', 'ST_Slope'], dtype='object') --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0


-------------------- 標準化されたdf_num --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0.256306,0.520852,1.272672,1.323176,1.742910,0.512265,1.252198,1.959903,0.913392,-0.484371,-0.549756,-0.183892,-0.481919,0.785575,-0.489267,-0.287456,1.015700,-0.871151
1,0.151431,-1.919930,-0.116784,1.796972,-0.573753,0.512265,-0.798596,-0.809353,-1.094820,2.064533,-0.549756,-0.183892,2.075039,-1.272953,-0.489267,-0.287456,-0.984543,1.147907
2,-0.687567,0.520852,-1.228348,-1.792941,1.742910,-0.706992,1.252198,-0.809353,-1.094820,-0.484371,1.818990,-0.183892,-0.481919,0.785575,-0.489267,-0.287456,1.015700,-0.871151


-------------------- 分割されたデータShape --------------------
x_train: (449, 18) x_test: (193, 18)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.9175946547884187
-------------------- 正解率（検証データ） --------------------
0.8704663212435233


In [9]:
df = pd.read_csv('./data/train.csv')
pipe = PipeLine()
pipe(df)
one_hot_columns = pipe.df_cat.columns  # カテゴリデータのカラムを取得

# それぞれのカラムがどのように影響するか調べていく
for cat in one_hot_columns:
    print('#'*150)
    pipe = PipeLine()  # インスタンスの初期化
    pipe(df) 
    pipe.one_hot(cat)
    pipe.standard_scaler()
    pack = pipe.fold_out_split(test_size=0.3)

    # サポートベクターマシンでモデルを訓練
    model = SVC()
    model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

    # 予測値を出力
    out_put(model, *pack)
    print()

######################################################################################################################################################
-------------------- ワンホットされたカラムChestPainType --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ASY,ATA,NAP,TA
0,56,1,155,342,1,150,1,3.0,1,0,0,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0
2,47,1,110,0,1,120,1,0.0,0,0,1,0


-------------------- 標準化されたdf_num --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ASY,ATA,NAP,TA
0,0.256306,0.520852,1.272672,1.323176,1.742910,0.512265,1.252198,1.959903,0.913392,-0.484371,-0.549756,-0.183892
1,0.151431,-1.919930,-0.116784,1.796972,-0.573753,0.512265,-0.798596,-0.809353,-1.094820,2.064533,-0.549756,-0.183892
2,-0.687567,0.520852,-1.228348,-1.792941,1.742910,-0.706992,1.252198,-0.809353,-1.094820,-0.484371,1.818990,-0.183892


-------------------- 分割されたデータShape --------------------
x_train: (449, 12) x_test: (193, 12)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.8685968819599109
-------------------- 正解率（検証データ） --------------------
0.844559585492228

######################################################################################################################################################
-------------------- ワンホットされたカラムRestingECG --------------------


...

-------------------- 分割されたデータShape --------------------
x_train: (449, 11) x_test: (193, 11)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.8797327394209354
-------------------- 正解率（検証データ） --------------------
0.8031088082901554

######################################################################################################################################################
-------------------- ワンホットされたカラムST_Slope --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,Down,Flat,Up
0,56,1,155,342,1,150,1,3.0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,1,0


-------------------- 標準化されたdf_num --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,Down,Flat,Up
0,0.256306,0.520852,1.272672,1.323176,1.742910,0.512265,1.252198,1.959903,-0.287456,1.015700,-0.871151
1,0.151431,-1.919930,-0.116784,1.796972,-0.573753,0.512265,-0.798596,-0.809353,-0.287456,-0.984543,1.147907
2,-0.687567,0.520852,-1.228348,-1.792941,1.742910,-0.706992,1.252198,-0.809353,-0.287456,1.015700,-0.871151


-------------------- 分割されたデータShape --------------------
x_train: (449, 11) x_test: (193, 11)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.9020044543429844
-------------------- 正解率（検証データ） --------------------
0.8860103626943006



# 分類木による評価

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
df = pd.read_csv('./data/train.csv')
# 前処理
pipe = PipeLine()
pipe(df)
one_hot_columns = pipe.df_cat.columns
pipe.one_hot(one_hot_columns)
pipe.standard_scaler()  # 標準化の処理を追加
pack = pipe.fold_out_split(test_size=0.3)

# 決定木のモデルを訓練
model = DecisionTreeClassifier(max_depth=3)
model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

# 予測値を出力
out_put(model, *pack)
print('-'*20, ' 説明変数の重要度 ', '-'*20)
display(pd.DataFrame([model.feature_importances_], columns=pipe.df_num.columns))
print()

-------------------- ワンホットされたカラムIndex(['ChestPainType', 'RestingECG', 'ST_Slope'], dtype='object') --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0


-------------------- 標準化されたdf_num --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0.256306,0.520852,1.272672,1.323176,1.742910,0.512265,1.252198,1.959903,0.913392,-0.484371,-0.549756,-0.183892,-0.481919,0.785575,-0.489267,-0.287456,1.015700,-0.871151
1,0.151431,-1.919930,-0.116784,1.796972,-0.573753,0.512265,-0.798596,-0.809353,-1.094820,2.064533,-0.549756,-0.183892,2.075039,-1.272953,-0.489267,-0.287456,-0.984543,1.147907
2,-0.687567,0.520852,-1.228348,-1.792941,1.742910,-0.706992,1.252198,-0.809353,-1.094820,-0.484371,1.818990,-0.183892,-0.481919,0.785575,-0.489267,-0.287456,1.015700,-0.871151


-------------------- 分割されたデータShape --------------------
x_train: (449, 18) x_test: (193, 18)
y_train: (449,) y_test: (193,)
-------------------- 正解率（訓練データ） --------------------
0.8797327394209354
-------------------- 正解率（検証データ） --------------------
0.8549222797927462
--------------------  説明変数の重要度  --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0.0,0.007564,0.0,0.0,0.0,0.032009,0.0,0.101312,0.203358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.655757


In [12]:
df = pd.read_csv('./data/train.csv')
pipe = PipeLine()
pipe(df)
one_hot_columns = pipe.df_cat.columns  # カテゴリデータのカラムを取得

# それぞれのカラムがどのように影響するか調べていく
for cat in one_hot_columns:
    print('#'*150)
    pipe = PipeLine()  # インスタンスの初期化
    pipe.viewer = False  # 前処理の過程を非表示にできる
    pipe(df) 
    pipe.one_hot(cat)
    pipe.standard_scaler()
    pack = pipe.fold_out_split(test_size=0.3)

    model = DecisionTreeClassifier(max_depth=3)
    model.fit(pack[0], pack[2])  # x_train, x_test, y_train, y_test = pack

    # 予測値を出力
    out_put(model, *pack)
    print('-'*20, ' 説明変数の重要度 ', '-'*20)
    display(pd.DataFrame([model.feature_importances_], columns=pipe.df_num.columns))
    print()

######################################################################################################################################################
-------------------- 正解率（訓練データ） --------------------
0.8151447661469933
-------------------- 正解率（検証データ） --------------------
0.772020725388601
--------------------  説明変数の重要度  --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ASY,ATA,NAP,TA
0,0.0,0.087394,0.0,0.081464,0.0,0.033667,0.0,0.172791,0.624684,0.0,0.0,0.0



######################################################################################################################################################
-------------------- 正解率（訓練データ） --------------------
0.8173719376391982
-------------------- 正解率（検証データ） --------------------
0.7668393782383419
--------------------  説明変数の重要度  --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,LVH,Normal,ST
0,0.077465,0.0,0.0,0.263856,0.028652,0.090309,0.532049,0.007669,0.0,0.0,0.0



######################################################################################################################################################
-------------------- 正解率（訓練データ） --------------------
0.8596881959910914
-------------------- 正解率（検証データ） --------------------
0.8497409326424871
--------------------  説明変数の重要度  --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,Down,Flat,Up
0,0.0,0.030911,0.0,0.109583,0.030789,0.057872,0.0,0.075718,0.0,0.0,0.695127
